In [1]:
import openai
import json
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
from credentials import OPENAPI_KEY
openai.api_key = OPENAPI_KEY

In [ ]:
train_ds_path = "DS_PATH/train.jsonl"
openai.File.create(file=open(train_ds_path, "rb"), purpose="fine-tune")

In [ ]:
openai.FineTuningJob.create(training_file="FILE_ID", model="gpt-3.5-turbo")

In [ ]:
openai.FineTuningJob.retrieve("FINETUNE_ID")

In [40]:
test_ds_path = "DS_PATH/test.jsonl"
with open(test_ds_path, "r") as f:
    test_dataset = [json.loads(line) for line in f]

In [41]:
test_dataset[0]["messages"]

[{'role': 'system',
  'content': 'vrl-gpt3.5-turbo is a chatbot that can give the sentiment of Turkish texts.'},
 {'role': 'user',
  'content': 'What is the sentiment of this Turkish text "Ulan Wifi\'ye bağlıyım ben. Ona bağlıyken Turkcell internet paketin bitti diye nasıl mesaj atabilir bana ya? Onu da mı ödeyelim"?'},
 {'role': 'assistant', 'content': 'negative'}]

In [ ]:
preds = []
refs = []
for example in tqdm(test_dataset):
    completion = openai.ChatCompletion.create(
        model="MODEL_ID",
        messages=example["messages"][:2]
        )
    model_output = completion.choices[0].message
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output["content"])
    refs.append(ground_truth)

In [122]:
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv("OUTPUT_PATH/output.csv", index=False)

In [2]:
temp_df  = pd.read_csv("OUTPUT_PATH/output.csv")
temp_df.head()

,preds,refs
0,negative,positive
1,neutral,positive
2,negative,positive
3,neutral,positive
4,neutral,positive


In [3]:
temp_df["preds"].value_counts()

preds
negative    1196
neutral     1001
positive     801
requests       1
neural         1
Name: count, dtype: int64

In [4]:
refs, preds = temp_df["refs"], temp_df["preds"]
output_labels = ["negative", "neutral", "positive"]
for index in range(len(preds)):
    if preds[index] not in output_labels:
        preds[index] = "neutral" 

In [5]:
print(classification_report(refs, preds, digits=4))

              precision    recall  f1-score   support

    negative     0.7642    0.7102    0.7362      1287
     neutral     0.5264    0.5517    0.5388       957
    positive     0.5743    0.6085    0.5909       756

    accuracy                         0.6340      3000
   macro avg     0.6216    0.6235    0.6220      3000
weighted avg     0.6405    0.6340    0.6366      3000

